In [1]:
import topycal
import os
import glob
import random

INSTR_PATH = os.path.join(os.getcwd(),"afi_txt")

In [2]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed
def do_fn_on_iter(fn, iterator, num_threads=6):
    futures = []
    if isinstance(num_threads, str):
        num_threads = int(num_threads)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for elem in iterator:
            futures.append(executor.submit(fn, elem))
    results = []
    for x in as_completed(futures):
        results.append(x.result())
    return results


In [3]:
def get_file_list(limit=500, shuffle=True):
    files = glob.glob("{}/afi*.txt".format(INSTR_PATH))
    if shuffle:
        random.shuffle(files)
    if limit:
        return files[0:limit]
    else:
        return files
    #data = myfile.read()
    
def read_file(fname):
    with open(fname, errors='replace') as fd:
        return fd.read()

In [4]:
file_list = get_file_list(limit=None)


In [5]:
import re
import os

def load_file(fname):       
    with open(fname, 'r') as myfile:
        contents = re.sub(r'[\t\n\r\x0b\x0c]',' ', myfile.read())
        return (os.path.basename(fname),re.sub("\s+",' ', contents))

def load_corpus(file_list):
    return {f[0]:f[1] for f in do_fn_on_iter(load_file, file_list)}    

In [6]:
corpus_dict = load_corpus(file_list)

In [7]:
import spacy
nlp = spacy.load("en_core_web_md")

For each document, we sentence segment then create a basket based on the 64bit murmurhash (after whitespace trimming and checking for a minimum sentence threshold).

In [8]:
import mmh3

from tqdm import tqdm
from collections import defaultdict

MIN_SENT_LEN = 15

def do_market_basket_analysis(doc_dict):
    baskets = defaultdict(set)
    for k,v in tqdm(doc_dict.items()):
        for sent in nlp(v).sents:
            if len(sent) > MIN_SENT_LEN:
                baskets[mmh3.hash64(sent.text.strip())].add(k)
    return baskets

In [9]:
baskets = do_market_basket_analysis(corpus_dict)

100%|██████████| 1141/1141 [22:28<00:00,  1.18s/it]


Each basket is trimmed based on being a certain minimum size.. logic here is to reduce noise.

In [10]:
# trim baskets

def trim_baskets(baskets, min_size=3):
    return {k:v for k,v in tqdm(baskets.items()) if len(v) > min_size}


In [11]:
trimmed_baskets = trim_baskets(baskets, min_size=4)

100%|██████████| 509336/509336 [00:00<00:00, 1319543.97it/s]


In [12]:
len(list(baskets.items()))

509336

In [13]:
len(list(trimmed_baskets.items()))

3030

Now we build a graph from the baskets, each combination of documents in each bucket is related to each other.. and the edge weight between documents is based on how many buckets they share. We also determine series here and enrich the node with that, so we can compare how links within or across series work.

In [14]:
import networkx as nx
from itertools import combinations
# Build a weighted graph from the baskets

def traverse_baskets(baskets):   
    graph = nx.Graph()
    for k,v in baskets.items():
        combs = combinations(v, 2)
        for comb in combs:
            # ensure nodes exist
            if comb[0] not in graph:
                graph.add_node(comb[0], series=comb[0].split('-')[0][-2:]) # split on '-' and take the last 2 chars from first part of split result
            if comb[1] not in graph:
                graph.add_node(comb[1], series=comb[1].split('-')[0][-2:]) # e.g. afi_12-21 -> 12; afi12-22 -> 12
            
            # if already exists, increment
            if comb[0] in graph[comb[1]]:
                curr_wt = graph.edges[comb[0], comb[1]]['weight']
                series_same = comb[0].split('-')[0][-2:] == comb[1].split('-')[0][-2:]
                graph.add_edge(comb[0],comb[1], weight=curr_wt+1, label="{}-{}".format(comb[0],comb[1]),sameseries="{}".format(series_same))
            # create new link
            else:
                series_same = comb[0].split('-')[0][-2:] == comb[1].split('-')[0][-2:]
                graph.add_edge(comb[0],comb[1], weight=1, label="{}-{}".format(comb[0],comb[1]), sameseries="{}".format(series_same))
    return graph
            
        

In [15]:
graph = traverse_baskets(trimmed_baskets)

In [16]:
len(graph.edges())

111646

More noise reduction.. prune nodes/edges that only share 1 bucket.

In [17]:
# Trim graphs with only 1 link
def prune_graph(graph):
    removal_candidates = []
    for edge in graph.edges():
        if graph.edges()[edge]['weight'] < 2:
            removal_candidates.append((edge[0],edge[1]))
    # remove low weight edges
    graph.remove_edges_from(removal_candidates)
    # remove isolate nodes
    graph.remove_nodes_from(list(nx.isolates(graph)))
    return graph

In [18]:
pruned_graph = prune_graph(graph)

In [19]:
len(pruned_graph.edges())

33631

Save as GML

In [20]:
nx.write_gml(pruned_graph,'afi_mba_pruned.gml')

We can also use the trimmed basket keys as a starting point for sentences that are very common in the corpus.. now if we remove those and then perform a conventional cosine similarity or other affinity analysis; we are effectively excluding the facsimile sentences. Again we exclude shorter sentences to reduce noise from outlines, etc.

In [21]:
redundant_sent_hashes = list(trimmed_baskets.keys())

In [22]:
import mmh3

from tqdm import tqdm
from collections import defaultdict

MIN_SENT_LEN = 15

def prune_corpus_of_reused_sentences(doc_dict, bad_hashes):
    pruned_dict = {}
    for docname,doctxt in tqdm(doc_dict.items()):
        reduced_sents = []
        for sent in nlp(doctxt).sents:
            if len(sent) > MIN_SENT_LEN:
                # check if hash is okay
                this_sent = sent.text.strip()
                if mmh3.hash64(this_sent) not in bad_hashes:
                    reduced_sents.append(this_sent)
        pruned_dict[docname] = " ".join(reduced_sents)
    return pruned_dict

In [23]:
pruned_dict = prune_corpus_of_reused_sentences(corpus_dict,redundant_sent_hashes)

100%|██████████| 1141/1141 [23:23<00:00,  1.23s/it]


In [24]:
list(pruned_dict.keys())[3]

'afi14-2crcv2.txt'

In [25]:
len(corpus_dict[list(pruned_dict.keys())[5]]) != len(pruned_dict[list(pruned_dict.keys())[5]])

True

Simple code to compute dot product on 1 docs using tfidf

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compare_docs(doc1,doc2):
    tfidf = TfidfVectorizer().fit_transform([doc1, doc2])
    diffs = list((tfidf * tfidf.T).A)
    return diffs[0][1]


Build similarity graph .. link each doc with similarity as attribute

In [27]:
import random
import networkx as nx

from itertools import combinations
# Build a weighted graph from the pruned docs based on cosine similarity

# O(n^2)!!
# should do this more efficiently.. fingerprint method or self-organizing map

def make_similarity_graph(doc_dict,sample=None):   
    graph = nx.Graph()
    pairwise_docs = list(combinations([k for k in doc_dict.keys()],2))
    if sample:
        random.shuffle(pairwise_docs)
        pairwise_docs = pairwise_docs[0:sample]
    
    for doc_key in doc_dict.keys():
        graph.add_node(doc_key, series=doc_key.split('-')[0][-2:])
    for pair in tqdm(pairwise_docs):
        series_same = pair[0].split('-')[0][-2:] == pair[1].split('-')[0][-2:]
        try:
            graph.add_edge(pair[0],pair[1],weight=compare_docs(doc_dict[pair[0]],doc_dict[pair[1]]),label="{}${}".format(pair[0],pair[1]), sameseries=series_same)
        except ValueError as e: # happens if we get empty vocab.. no worries
            print("{e}")
            pass
    return graph
            
        

In [28]:
key = list(pruned_dict.keys())[1]
print(len(pruned_dict[key]))
print(len(corpus_dict[key]))

25322
31611


In [29]:
print(corpus_dict[key])

 BY ORDER OF THE SECRETARY OF THE AIR FORCE AIR FORCE INSTRUCTION 36-2906 30 MAY 2013 Personnel PERSONAL FINANCIAL RESPONSIBILITY COMPLIANCE WITH THIS PUBLICATION IS MANDATORY ACCESSIBILITY: Publications and forms are available on the e-Publishing website at www.e-Publishing.af.mil for downloading or ordering. RELEASABILITY: There are no releasability restrictions on this publication OPR: AFPC/DPSIM Supersedes: AFI 36-2906, 1 January 1998 Certified by: AF/A1S (Brig Gen Eden J. Murrie) Pages: 12 This instruction implements Air Force Policy Directive (AFPD) 36-29, Military Standards, Department of Defense Instruction (DoDI) 1344.09, Indebtedness of Military Personnel, and DoDI 1344.12, Indebtedness Processing Procedures for Military Personnel. This instruction is applicable to the total force to include Active Duty, Air Force Reserve, and Air National Guard members. Refer recommended changes and questions about this publication to the Office of Primary Responsibility (OPR), using AF Form

In [30]:
print(pruned_dict[key])

BY ORDER OF THE SECRETARY OF THE AIR FORCE AIR FORCE INSTRUCTION 36-2906 30 MAY 2013 Personnel AFPC/DPSIM Supersedes: AFI 36-2906, 1 January 1998 Certified by: AF/A1S (Brig Gen Eden J. Murrie) This instruction implements Air Force Policy Directive (AFPD) 36-29, Military Standards, Department of Defense Instruction (DoDI) 1344.09, Indebtedness of Military Personnel, and DoDI 1344.12, Indebtedness Processing Procedures for Military Personnel. This instruction is applicable to the total force to include Active Duty, Air Force Reserve, and Air National Guard members. Refer recommended changes and questions about this publication to the Office of Primary Responsibility (OPR), using AF Form 847, Recommendation for Change of Publication; route AF Form 847s from the field through the appropriate functional’s chain of command. This publication may be supplemented at any level, but all direct supplements must be routed to the OPR of this publication for coordination prior to the certification an

In [31]:
similarity_graph = make_similarity_graph(pruned_dict,sample=None)

 16%|█▌        | 100825/650370 [35:36<3:14:06, 47.19it/s]

{e}


 16%|█▌        | 101040/650370 [35:39<3:13:51, 47.23it/s]

{e}


 16%|█▌        | 101183/650370 [35:41<3:13:41, 47.25it/s]

...

 94%|█████████▎| 608416/650370 [3:36:11<14:54, 46.90it/s]

{e}


 98%|█████████▊| 634744/650370 [3:46:58<05:35, 46.61it/s]

{e}


100%|██████████| 650370/650370 [3:52:36<00:00, 46.60it/s]


In [32]:
doc_a, doc_b = list(similarity_graph.edges())[3]
similarity_graph.edges()[list(similarity_graph.edges())[3]]['weight'] == compare_docs(pruned_dict[doc_a],pruned_dict[doc_b])

True

In [33]:
# Trim graphs lower similarity
def prune_similarity_graph(graph,floor=0.4):
    removal_candidates = []
    for edge in graph.edges():
        if graph.edges()[edge]['weight'] < floor:
            removal_candidates.append((edge[0],edge[1]))
    # remove low weight edges
    graph.remove_edges_from(removal_candidates)
    # remove isolate nodes
    graph.remove_nodes_from(list(nx.isolates(graph)))
    return graph

In [35]:
#pruned_similarity = prune_similarity_graph(similarity_graph,floor=0.85)

In [37]:
#nx.write_gml(pruned_similarity,'afi_cosine_pruned_85thresh.gml')